<a href="https://colab.research.google.com/github/sj0054pe/MBTI_Estim/blob/master/src/5A_S9_BERT%E7%94%A8%E3%83%87%E3%83%BC%E3%82%BF%E3%82%BB%E3%83%83%E3%83%88_U20R30000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<center>BERT データセットの作成 S9</center>

###<center>リツイート・@mention・URLの除去・短い文(20単語未満)の削除</center>

###<center>各尺度で3万件ずつ用意</center>


#0. 作業環境の準備

##0.1. google driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##0.2. カレントディレクトリへ移動

In [ ]:
%cd /content/drive/MyDrive/研究室/研究_M2/src

/content/drive/MyDrive/研究室/研究_M2/src


##0.3. 現在時刻の取得

In [ ]:
import datetime
import pytz

def pull_Datetime():
  DATETIME_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  DATETIME_now = DATETIME_now.strftime("%Y-%m-%d")
  #DATETIME_now = DATETIME_now.strftime("%Y/%m/%d %H:%M:%S")
  #print(DATETIME_now)

  return DATETIME_now

print(pull_Datetime())

2021-12-08


In [ ]:
SEASONS='S9'
HIGHER ='20'
RAFFLE=30000

TERM='%s_W%s_R%s' % (SEASONS, HIGHER, RAFFLE)
print(TERM)

S9_W20_R30000


##option. データセットの指定

In [ ]:
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v7_all_2021-12-04_W20,csv'
INPUT_CSV_v5='./Assets/Assets_Output/Tweet_Datasets/v7_all_2021-12-04_W20.csv'

#1. データセットのロード

In [ ]:
import pandas as pd
df_v0=pd.read_csv('%s' % INPUT_CSV_v5, engine='python')

# データの確認
print(f'データサイズ： {df_v0.shape}')

df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

データサイズ： (133350, 14)


,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
1,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
2,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
3,"こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...",2017-11-09 14:32:30+00:00,INTJ,I,N,T,J
4,ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...,2017-11-07 16:36:08+00:00,INTJ,I,N,T,J


#2. データの加工

##2.1. 必要なデータ（文章とラベル）の抽出

In [ ]:
#df_v0.drop(['notes', 'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',  'Unnamed: 0.1.1.1'],axis=1, inplace=True)
df_v0.dropna(how='any', axis=0, inplace=True)

df_v0=df_v0
df_v0[['text', 'tweet_time', 'MBTI_Type','label_EorI','label_NorS','label_ForT','label_JorP']].head()

,text,tweet_time,MBTI_Type,label_EorI,label_NorS,label_ForT,label_JorP
0,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,INTJ,I,N,T,J
1,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,INTJ,I,N,T,J
2,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,INTJ,I,N,T,J
3,"こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...",2017-11-09 14:32:30+00:00,INTJ,I,N,T,J
4,ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...,2017-11-07 16:36:08+00:00,INTJ,I,N,T,J


In [ ]:
df_v3 = df_v0
#df_v3=df.sample(n=100)
df_v3[['text', 'tweet_time', 'conversation_id','MBTI_Type']].head(5)

,text,tweet_time,conversation_id,MBTI_Type
0,CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？,2017-12-29 14:01:24+00:00,9.467430e+17,INTJ
1,中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...,2017-11-26 06:54:54+00:00,9.346768e+17,INTJ
2,Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。,2017-11-19 16:00:34+00:00,9.322774e+17,INTJ
3,"こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...",2017-11-09 14:32:30+00:00,9.286314e+17,INTJ
4,ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...,2017-11-07 16:36:08+00:00,9.279377e+17,INTJ


In [ ]:
df_v3['text'].head(5)

0        CLIP STUDIO TABMATEがコミケで販売されたそうだけど買った方いるのかな！？
1    中小の劇団のチケットって、安すぎたら箱や演者やスタッフさんのギャラが払えない。かと言って高く...
2              Vステなら大丈夫だったのになぁ… まぁQちゃんの事だからなんとかなるでしょう。
3    こんなにカッコよくキメてたのに…#buzzup BOOSTER - KEN THE 390,...
4    ブーメランパンツはあなたの息子と一緒にクリスマスの一夜を過ごします。可愛がってくださいね。 ...
Name: text, dtype: object

In [ ]:
df_v5=df_v3

df_v5_E_1=df_v5[df_v5['label_EorI']=='E']
df_v5_I_1=df_v5[df_v5['label_EorI']=='I']
df_v5_E_2=df_v5_E_1.sample(n=15000)
df_v5_I_2=df_v5_I_1.sample(n=15000)
df_v5_EorI=pd.concat([df_v5_E_2, df_v5_I_2])

df_v5_N_1=df_v5[df_v5['label_NorS']=='N']
df_v5_S_1=df_v5[df_v5['label_NorS']=='S']
df_v5_N_2=df_v5_N_1.sample(n=15000)
df_v5_S_2=df_v5_S_1.sample(n=15000)
df_v5_NorS=pd.concat([df_v5_N_2, df_v5_S_2])

df_v5_F_1=df_v5[df_v5['label_ForT']=='F']
df_v5_F_2=df_v5_F_1.sample(n=15000)
df_v5_T_1=df_v5[df_v5['label_ForT']=='T']
df_v5_T_2=df_v5_T_1.sample(n=15000)
df_v5_ForT=pd.concat([df_v5_F_2, df_v5_T_2])

df_v5_J_1=df_v5[df_v5['label_JorP']=='J']
df_v5_P_1=df_v5[df_v5['label_JorP']=='P']
df_v5_J_2=df_v5_J_1.sample(n=15000)
df_v5_P_2=df_v5_P_1.sample(n=15000)
df_v5_JorP=pd.concat([df_v5_J_2, df_v5_P_2])

In [ ]:
def num_Type_Index(element):
  num_Users_Type=df_v5_EorI.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()

  print(num_Users_Type.size())
  print()

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

label_EorI
E    15000
I    15000
dtype: int64

label_NorS
N    16410
S    13590
dtype: int64

label_ForT
F    19114
T    10886
dtype: int64

label_JorP
J    10881
P    19119
dtype: int64



#3. データの保存

##3.1. 手法4までで作成した投稿を更新

##3.2. データセットS3の作成

In [ ]:
theDate=pull_Datetime()
df_v5_EorI.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_EorI.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_EorI.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()
df_v5_NorS.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_NorS.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_NorS.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()
df_v5_ForT.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_ForT.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_EorI.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()
df_v5_JorP.to_csv('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_JorP.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print('./Assets/Assets_Output/Tweet_Datasets/v6%s_forBERT_%s_%s_%s_JorP.csv' % (SEASONS, theDate, HIGHER, RAFFLE))
print()

./Assets/Assets_Output/Tweet_Datasets/v6S9_forBERT_2021-12-06_20_30000_EorI.csv
./Assets/Assets_Output/Tweet_Datasets/v6S9_forBERT_2021-12-06_20_30000_NorS.csv
./Assets/Assets_Output/Tweet_Datasets/v6S9_forBERT_2021-12-06_20_30000_EorI.csv
./Assets/Assets_Output/Tweet_Datasets/v6S9_forBERT_2021-12-06_20_30000_JorP.csv


##3.3. 記録用に各指標のユーザー数を表示

In [ ]:
def num_Type_Index(element):
  num_Users_Type=df_v5_EorI.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()
for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)


def num_Type_Index(element):
  num_Users_Type=df_v5_NorS.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()
for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

def num_Type_Index(element):
  num_Users_Type=df_v5_ForT.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()
for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

def num_Type_Index(element):
  num_Users_Type=df_v5_JorP.groupby(['%s' % element])
  num_Users_Type.groups
  num_Users_Type.size()
  print(num_Users_Type.size())
  print()

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_Type_Index(element)

label_EorI
E    15000
I    15000
dtype: int64

label_NorS
N    16382
S    13618
dtype: int64

label_ForT
F    19146
T    10854
dtype: int64

label_JorP
J    10843
P    19157
dtype: int64

label_EorI
E     4814
I    25186
dtype: int64

label_NorS
N    15000
S    15000
dtype: int64

label_ForT
F    20428
T     9572
dtype: int64

label_JorP
J    11254
P    18746
dtype: int64

label_EorI
E     5062
I    24938
dtype: int64

label_NorS
N    17852
S    12148
dtype: int64

label_ForT
F    15000
T    15000
dtype: int64

label_JorP
J    10588
P    19412
dtype: int64

label_EorI
E     4740
I    25260
dtype: int64

label_NorS
N    16906
S    13094
dtype: int64

label_ForT
F    20428
T     9572
dtype: int64

label_JorP
J    15000
P    15000
dtype: int64



#Option 最大単語数の計算

In [ ]:
!pip install transformers

!pip install fugashi
!pip install ipadic

In [ ]:
## Tokenizerの準備
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
from tqdm.notebook import tqdm
import numpy as np

def num_max_words(TYPE_INDEX):
  sentences= df_S['text'].values #sentences = df.sentence.values
  labels= df_S['%s' % TYPE_INDEX].values #labels = df.label.values

  # 最大単語数の確認
  max_len = []
  # 1文づつ処理
  len_sentences = len(sentences)
  for sent in tqdm(sentences):
      # Tokenizeで分割
      token_words = tokenizer.tokenize(sent)
      # 文章数を取得してリストへ格納
      max_len.append(len(token_words))
  # 最大の値を確認
  len_max=max(max_len)
  #len_q0, len_q25, len_q50, len_q75, len_q100=np.percentile(max_len,  [0, 25, 50, 75, 100])
  #len_q75=int(len_q75)
  print('最大単語数: ', max(max_len))

for element in ["label_EorI", "label_NorS", "label_ForT", "label_JorP"] :
  num_max_words(element)